# **Google Colab TPU Kulanarak MNIST Eğitimi**


---


**Kaynak:** *[Jupyter NBviewer](https://nbviewer.jupyter.org/github/cedrickchee/data-science-notebooks/blob/master/notebooks/tensorflow/google_cloud_tpu/guide_to_tensorflow_keras_on_tpu_mnist.ipynb#Inference)*

[<img align="right" width="100" height="100" src="http://www.i2symbol.com/images/symbols/style-letters/circled_latin_capital_letter_a_u24B6_icon_128x128.png">](https://www.ayyucekizrak.com/)


In [0]:
import numpy as np

import tensorflow as tf
import time
import os

import tensorflow.keras
from tensorflow.keras.datasets import mnist, fashion_mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten,Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

In [0]:
print(tf.__version__)
print(tf.keras.__version__)

1.12.0-rc1
2.1.6-tf


## TPU Kontrolü
"Runtime"  ve "Change Runtime Type" sekmelerini kullanarak TPU seçimini yapıp kaydedin.

**TPU_ADDRESS** değişkenine ihtiyaç olacak. Aşağıdaki şekilde TPU  aktivasyonunu sağlamak mümkün. Kod parçası çalıştırıldığında 

`Found TPU at: grpc://10.41.118.242:8470` şeklinde TPU bulundu çıkışı vermesi gerek!

In [0]:
try:
  device_name = os.environ['COLAB_TPU_ADDR']
  TPU_ADDRESS = 'grpc://' + device_name
  print('Found TPU at: {}'.format(TPU_ADDRESS))

except KeyError:
  print('TPU not found')

Found TPU at: grpc://10.41.118.242:8470


### Standart MNIST işlemleri

In [0]:
batch_size = 1024
num_classes = 10
epochs = 5
learning_rate = 0.001

# Giriş görüntüsü boyutları
img_rows, img_cols = 28, 28

# Veri kümesi eğitim ve test kümesi olarak iki kısıma ayrılır.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [0]:
# Sınıf vektörlerini ikili sınıf matrislerine dönüştürme işlemi 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

### `tf.data` Kullanımı
 TPU'larda fixed şekil oluşması için `drop_remainder = True` olarak yazıldığına emin olmak gerek.

In [0]:
def train_input_fn(batch_size=1024):
  # Girişleri bir veri kümesine dönüştür. 
  dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train))

  # Karıştır, tekrar et ve batch (küme) örnekleri 
  dataset = dataset.shuffle(1000).repeat().batch(batch_size, drop_remainder=True)

  # veri kümesine.
  return dataset

In [0]:
def test_input_fn(batch_size=1024):
  # Girişleri bir veri kümesine dönüştür. .
  dataset = tf.data.Dataset.from_tensor_slices((x_test,y_test))

  # sKarıştır, tekrar et ve batch (küme) örnekleri 
  dataset = dataset.shuffle(1000).repeat().batch(batch_size, drop_remainder=True)

  # veri kümesine.
  return dataset

## Model Tanımlama
TPU'lar (ve XLA) sabit şekiller gerektirdiğinden, giriş şekli ve küme (batch) boyutta geçmek gerekir.

Modelin geri kalanı bilinen Evrişimli Sinir Ağı (CNN)

In [0]:
Inp = tf.keras.Input(
    name='input', shape=input_shape, batch_size=batch_size, dtype=tf.float32)

x = Conv2D(32, kernel_size=(3, 3), activation='relu',name = 'Conv_01')(Inp)
x = MaxPooling2D(pool_size=(2, 2),name = 'MaxPool_01')(x)
x = Conv2D(64, (3, 3), activation='relu',name = 'Conv_02')(x)
x = MaxPooling2D(pool_size=(2, 2),name = 'MaxPool_02')(x)
x = Conv2D(64, (3, 3), activation='relu',name = 'Conv_03')(x)
x = Flatten(name = 'Flatten_01')(x)
x = Dense(64, activation='relu',name = 'Dense_01')(x)
x = Dropout(0.5,name = 'Dropout_02')(x)

output = Dense(num_classes, activation='softmax',name = 'Dense_02')(x)

In [0]:
model = tf.keras.Model(inputs=[Inp], outputs=[output])

In [0]:
# Keras optimizer yerine tf optimizer kullanılmalıdır.

model.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['acc'])

## Keras Modelinde TPU Oluşturma/Çalıştırma
`tf.contrib.tpu.keras_to_tpu_model` kullanılması gerekiyor!

8 çekirdek bir TPU olduğunu ekrana yazdırır.

Toplam 1024 işlem için, çekirdek başına 128 adetlik batchler kullanılır.

128, 256, 512 vb. şeklinde de kullanılabilir.

In [0]:
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)))

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.41.118.242:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 643035114549247129)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 17159233767363578909)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 16798713706203804527)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 4472406307199707535)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 5275484491368911105)
INFO:tensorflow:*** Available Device: _Device

In [0]:
tpu_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (1024, 28, 28, 1)         0         
_________________________________________________________________
Conv_01 (Conv2D)             (1024, 26, 26, 32)        320       
_________________________________________________________________
MaxPool_01 (MaxPooling2D)    (1024, 13, 13, 32)        0         
_________________________________________________________________
Conv_02 (Conv2D)             (1024, 11, 11, 64)        18496     
_________________________________________________________________
MaxPool_02 (MaxPooling2D)    (1024, 5, 5, 64)          0         
_________________________________________________________________
Conv_03 (Conv2D)             (1024, 3, 3, 64)          36928     
_________________________________________________________________
Flatten_01 (Flatten)         (1024, 576)               0         
__________

### Eğitim işleminde  `tf.data` kullanımı
MNIST veri kümesini TPU üzerinde eğitmeye çalışmak TPU kullanmak için iyi bir başlangıç!

In [0]:
tpu_model.fit(
  train_input_fn,
  steps_per_epoch = 60,
  epochs=10,
)

Epoch 1/10
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(1024,), dtype=tf.int32, name=None), TensorSpec(shape=(1024, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(1024, 10), dtype=tf.float32, name=None)]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 2.001673698425293 secs
INFO:tensorflow:Setting weights on TPU model.
60/60 [==============================] - 7s 120ms/step - loss: 0.8985 - acc: 0.7165
Epoch 2/10
60/60 [==============================] - 3s 43ms/step - loss: 0.2113 - acc: 0.9387
Epoch 3/10
60/60 [==============================] - 3s 44ms/step - loss: 0.1290 - acc: 0.9629
Epoch 4/10
60/60 [==============================] - 3s 45ms/step - loss: 0.0955 - acc: 0.9729
Epoch 5/10
60/60 [==============================] - 3s 43ms/step - loss: 0.0796 - acc: 0.9770
Epoch 6/10
60/60 [=====

In [0]:
tpu_model.save_weights('./MNIST_TPU_1024.h5', overwrite=True)

INFO:tensorflow:Copying TPU weights to the CPU


## Sonuç
Modeli değerlendirmek.

In [0]:
tpu_model.evaluate(test_input_fn, steps = 100)

INFO:tensorflow:New input shapes; (re-)compiling: mode=eval (# of cores 8), [TensorSpec(shape=(1024,), dtype=tf.int32, name=None), TensorSpec(shape=(1024, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(1024, 10), dtype=tf.float32, name=None)]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 1.1425998210906982 secs
100/100 [==============================] - 7s 66ms/step


[0.022670583389699458, 0.992496337890625]